# 7.1 Create a crawler using the webcrawler provided and crawl 200 pages

#1 Install Scrapy, a framework for crawling websites.

In [1]:
!pip install scrapy

grin 1.2.1 requires argparse>=1.1, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import scrapy

Create a scrapy project

In [2]:
!scrapy startproject tutorial

New Scrapy project 'tutorial', using template directory 'c:\\users\\tonye iyalla\\anaconda2\\lib\\site-packages\\scrapy\\templates\\project', created in:
    C:\Users\Tonye Iyalla\Documents\BigData\big-data-python-class\Homeworks\Homework7\tutorial

You can start your first spider with:
    cd tutorial
    scrapy genspider example example.com


In [3]:
ls

 Volume in drive C is Local Disk
 Volume Serial Number is FE3C-B042

 Directory of C:\Users\Tonye Iyalla\Documents\BigData\big-data-python-class\Homeworks\Homework7

11/12/2018  07:36 PM    <DIR>          .
11/12/2018  07:36 PM    <DIR>          ..
11/11/2018  02:01 PM    <DIR>          .ipynb_checkpoints
11/12/2018  07:16 PM               867 amazon.py
11/12/2018  07:31 PM            12,811 Iyalla-HW7-linkanalysis-partFall2018.ipynb
11/12/2018  07:36 PM    <DIR>          tutorial
11/11/2018  02:40 PM             7,019 Untitled.ipynb
               3 File(s)         20,697 bytes
               4 Dir(s)  201,833,095,168 bytes free


cd into tutorial spiders directory

In [1]:
cd tutorial/tutorial/spiders

C:\Users\Tonye Iyalla\Documents\BigData\big-data-python-class\Homeworks\Homework7\tutorial\tutorial\spiders


In [2]:
import scrapy

In [7]:
!scrapy genspider amazon amazon.py

Spider 'amazon' already exists in module:
  tutorial.spiders.amazon


7.1 Create crawler and crawl webpage --amazon

In [ ]:
# %load amazon.py
import scrapy

class amazonItem(scrapy.Item):
    title = scrapy.Field()
    link = scrapy.Field()
    resp = scrapy.Field()
class AmazonSpider(scrapy.Spider):
    name = 'amaz'
    allowed_domains = ['https://www.amazon.com/']
    start_urls = ['https://www.amazon.com/books-used-books-textbooks/b/?ie=UTF8&node=283155&ref_=topnav_storetab_b','https://www.amazon.com/gp/browse.html?node=2625373011&ref_=nav_em_T1_0_4_14_1__mov', 'https://www.amazon.com/b/ref=s9_acss_bw_cg_TXTHPCCG_1c1_w?node=468204&pf_rd_m=ATVPDKIKX0DER&pf_rd_s=merchandised-search-8&pf_rd_r=F3C0AEBDPSV192J7BNP8&pf_rd_t=101&pf_rd_p=b48a6433-fbec-472a-9c4f-db43997d01de&pf_rd_i=465600']

    def parse(self, response):
        res = scrapy.Selector(response)
        titles = res.xpath('//ul/li')
        items = []
        for title in titles:
            item = nbaItem()
            item["title"] = title.xpath("a/text()").extract()
            item["link"] = title.xpath("a/@href").extract()
            item["resp"] = response
            if item["title"] != []:
                items.append(item)

        return items

In [ ]:
!scrapy crawl amaz -o amazon.csv

# 7.2. Stochastic matrix

Clean data, extra words surrounding the link e.g. response code 200 and titles

In [7]:
import pandas as pd
temp =[]
data=pd.read_csv("amazon.csv")
data['link']='http://www.amazon.com'+data['link']
x=data['resp'][0]
print x
temp.append((x.split()[-1]).split('>')[-2])
print temp
for i in range(len(data)):
    if data['resp'][i] == x:
        temp.append(data['link'][i])
    else:
        x=data['resp'][i]

<200 https://www.amazon.com/movies-tv-dvd-bluray/b?ie=UTF8&node=2625373011>
['https://www.amazon.com/movies-tv-dvd-bluray/b?ie=UTF8&node=2625373011']


In [37]:
amazon_result = list(pd.DataFrame(temp)[0].unique())
links = []
length = len(temp)
for i, val in enumerate(temp):
    if i < length-1:
        links.append((temp[i], temp[i+1]))

Create stochastic matrix

In [54]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
df = pd.DataFrame(index=amazon_result, columns=amazon_result)
m = df.replace(np.NaN, 0)
for i in links:
    m.loc[i] = 1.0
arr = np.array(m)
arrSum = arr.sum(axis=1)
res = arr/arrSum[:, np.newaxis]
matrixAr = np.nan_to_num(res)
matrixAr

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# 7.3. Page rank

In [17]:
from numpy import *
 
def pagerank(H):
    n= len(H)
    w = zeros(n)
    rho = 1./n * ones(n);
    for i in range(n):
        if multiply.reduce(H[i]== zeros(n)):
            w[i] = 1
    newH = H + outer((1./n * w),ones(n))
 
    theta=0.85
    G = (theta * newH) + ((1-theta) * outer(1./n * ones(n), ones(n)))
    print rho
    for j in range(10):
        rho = dot(rho,G)
        print rho

In [29]:
RankVal=pd.DataFrame(pageRank(matrixAr,s=.86))
RankSort = (RankVal.sort_values(0)).head(5)
RankList = list(RankSort.index)
print RankList

[0L, 1L, 2L, 72L, 3L]


In [34]:
urls = pd.DataFrame(amazon_result)
print "Top 5 page URLs:"
for i in RankList:
    print urls[0][i]

Top 5 page URLs:
https://www.amazon.com/movies-tv-dvd-bluray/b?ie=UTF8&node=2625373011
http://www.amazon.com/Movies/b?ie=UTF8&node=2649512011
http://www.amazon.com/tv-dvd-bluray/b?ie=UTF8&node=2649513011
http://www.amazon.com/gp/promotions/details/popup/AWT354OR7BM1U/?ie=UTF8&ref=MoviesHPBB_POPG
http://www.amazon.com/amazon-fire-tv-4k-uhd-streaming-media-player/dp/B01N32NCPM


# 7.4. Page Hits

In [31]:
def hits(A):
    n= len(A)
    Au= dot(transpose(A),A)
    Hu = dot(A,transpose(A))
    a = ones(n);
    h = ones(n)
    #print a,h
    for j in range(5):
        a = dot(a,Au)
        a= a/sum(a)
        h = dot(h,Hu)
        h = h/ sum(h)
    return h

In [32]:
hitVal = pd.DataFrame(hits(matrixAr))
hitSort = (hitVal.sort_values(0, ascending=False)).head(5)
hitList = list(hitSort.index)

In [33]:
urls = pd.DataFrame(amazon_result)
print "Top 5 page URLs:"
for i in hitList:
    print urls[0][i]

Top 10 page URLs:
http://www.amazon.com/b?ie=UTF8&node=11913537011
http://www.amazon.comhttps://www.amazon.com/l/ref=map_1_b2b_GW_FT?node=17882322011
http://www.amazon.com/Art-House-International-DVD/b?ie=UTF8&node=163313
http://www.amazon.com/Accessories-Books/b?ie=UTF8&node=265040
http://www.amazon.com/iss/credit/rewardscardmember?_encoding=UTF8&plattr=CBFOOT
